In [22]:
#import libraries and functions
import numpy as np
import scipy.linalg as la
import random

In [23]:
#Pauli matrices
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, -1]])

In [24]:
#setup inital hamiltonian
msize = 4
J=1
g=1
hinit = np.zeros((msize, msize), dtype=np.complex128)
hinit+=-J*np.kron(sz,sz)
hinit+=-g*np.kron(sx,np.eye(2))
hinit+=-g*np.kron(np.eye(2),sx)

In [25]:
#setup final hamiltonian; assuming lambda is in front of the x field
hfin = np.zeros((msize, msize), dtype=np.complex128)
hfin+=-J*np.kron(sz,sz)

In [26]:
tau = 10 #total trajectory time to evolve lambda from 0 to 1
dtau = 1.0 #time-step for trajectory
num_steps = int(tau/dtau)
beta = 4.0 #inverse temperature of systems

In [27]:
eigvals_i, eigvecs_i = la.eig(hinit) # eigenvalues of the initial state
eigvals_f, eigvecs_f = la.eig(hfin) # eigenvalues of the final

In [28]:
#calculate the initial partition function
zval_i = 0
for ival in eigvals_i:
    zval_i = zval_i +np.real(np.exp(-beta*ival))

#calculate the final partition function
zval_f = 0
for ival in eigvals_f:
    zval_f = zval_f +np.real(np.exp(-beta*ival))

In [29]:
# free energy difference to compare
print(zval_i,zval_f,-np.log(zval_f)+np.log(zval_i))

7718.483169746547 109.23293134406595 4.25789055617836


In [ ]:
work = []

In [ ]:
#define simulation variables
tau = 10 #total trajectory time to evolve lambda from 0 to 1
dtau = 1.0 #time-step for trajectory
num_steps = int(tau/dtau)
T = 1000 #total number of trajectories
dt =  dtau #timestep for Trotter approximation: setting equal to dtau means one trotter-step per time-step in evolution
lambda_protocol = np.linspace(1.0, 0,num_steps)
print(lambda_protocol)
dldt = (lambda_protocol[1]-lambda_protocol[0])/dtau # d(lambda)/d(tau)
print(dldt) 

In [38]:
#choose a random number
asv = random.uniform(0, 1)

#pick a state to propogate with the correct probability
select = 0
scum = 0
for ival in range(msize):
   scum = scum+np.real(np.exp(-beta*eigvals_i[ival]))/zval_i
   if scum > asv: select = ival
   if scum > asv: break

curstate =  eigvecs_i[:,select]

In [43]:
#iterate through our protocol--maybe just make this the full loop
v = curstate
o = np.kron(numpy.eye(2), sz) + np.kron(sz,numpy.eye(2))
work_i = 0
for i in range(num_steps):
    hnew = hinit-i*(-g*np.kron(sx,np.eye(2))+-g*np.kron(np.eye(2),sx))*num_steps
    w = la.expm(-1j*dtau*hnew) @ v
    v = w
    m = v.conjugate().T @ o @ v #measure the magnetization
    work_i += m*dtau*

In [ ]:
work.append(work_i)